In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2Model

/Users/sashafedorova/jetBrainsIntern/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Load the pretrained GPT2 model and tokenizer

In [2]:
model_name = "gpt2"
model = GPT2Model.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Input sequence

In [3]:
text = "Hello, how are you?"
input_ids = tokenizer.encode(text, return_tensors="pt")
attention_mask = torch.ones_like(input_ids)
with torch.no_grad():
    original_outputs = model(input_ids, attention_mask=attention_mask, output_attentions=True)


Print the original attention values for the first layer and first head


In [4]:
original_attention = original_outputs.attentions[0][0]
print("Original Attention:")
print(original_attention)

Original Attention:
tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [9.7093e-01, 2.9067e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [7.3162e-01, 1.4214e-01, 1.2624e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.9679e-01, 9.4215e-02, 1.3803e-01, 7.0965e-02, 0.0000e+00,
          0.0000e+00],
         [6.3546e-01, 8.1361e-02, 1.7181e-01, 4.5568e-02, 6.5797e-02,
          0.0000e+00],
         [5.8499e-01, 3.4026e-02, 1.9450e-01, 3.3753e-02, 1.0432e-01,
          4.8413e-02]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.7076e-03, 9.9329e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.2836e-03, 1.2506e-03, 9.9747e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.0240e-03, 1.0826e-02, 2.3372e-03, 9.8581e-01, 0.0000e+00,
          0.0000e+00],
         [1.3698e-02, 6.0359e-03, 8.2985e-03, 8.

Modify the GPT2 model with MeanGPT2Attention

In [5]:
from gpt2_attentions_aproximations import GPT2Wrapper

modified_model = GPT2Wrapper(model)
with torch.no_grad():
    modified_outputs = modified_model(input_ids, attention_mask=attention_mask)

modified_attention = modified_outputs.attentions[0][0]
print("Modified Attention:")
print(modified_attention)


IndexError: Dimension specified as -2 but tensor has no dimensions

In [8]:
# Calculate the mean of the original attention values
mean_original_attention = original_attention.mean(dim=-1)
print("Mean Original Attention:")
print(mean_original_attention)

# Check if the modified attention values match the mean of the original attention values
mean_modified_attention = modified_attention.mean(dim=-1)
mean_equal = torch.allclose(mean_modified_attention, mean_original_attention)
print("Mean Attention Equal:", mean_equal)

Mean Original Attention:
tensor([[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]])


NameError: name 'modified_attention' is not defined